In [1]:
degfiles = ['../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_de_results_her19_v_herm.txt',
            '../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_de_results_male_v_her19.txt',
            '../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_de_results_male_v_herm.txt']

expfile = '../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_combined_kallisto_normalizedcounts_rlog_matrix.txt'
catfile = 'gene_categories_v2.txt'
txtoutfile = 'gene_expression_supplemental.txt'
lfc_threshold = 0.58
fdr_threshold = 0.1
exp_threshold = 3



In [2]:
geneDict = {}

fi = open(catfile)
for line in fi:
    if line[0] == '#':
        continue
        
    #print(line)
    cat, color, path, kegg, genes = line.rstrip().split('\t')
    #print(gene)
    for gene in genes.split('; '):
        #print(gene, cat, path)
        geneDict[gene] = [cat, path]

fi.close()

In [3]:
degDict = {}

for degfile in degfiles:
    comparison = degfile.split('burow_de_results_')[1].split('.txt')[0]
    #print(comparison)
    degDict[comparison] = {}
    
    
    fi = open(degfile)
    
    for line in fi:
        #print(line)
        if line.startswith('baseMean'):
            continue
        gene, mean, lfc, lfcse, pval, padj = line.rstrip().split('\t')
        if gene in geneDict:
            #print(padj)
            degDict[comparison][gene] = [mean, lfc, lfcse, pval, padj]
    
    fi.close()

In [4]:
fi = open(expfile)
expDict = {}

for line in fi:
    if line[0] == 'F':
        continue
    col = line.rstrip().split('\t')
    gene = col.pop(0)
    normcounts = '\t'.join(col)
    #print(gene, tmps)
    
    expDict[gene] = normcounts

fi.close()

In [5]:
fo = open(txtoutfile, 'w')

fo.write('pathway\tgene family\tgene\t')
for comparison in degDict:
    fo.write(comparison + ' log2FC\t' + comparison + ' adjusted p-value\t')
fo.write('her19-1\ther19-2\ther19-3\tWT herm-1\tWT herm-2\tWT herm-3\tmale-1\tmale-2\tmale-3\n')


for gene in geneDict:
    path = geneDict[gene][1]
    cat = geneDict[gene][0]
    
    fo.write(path + '\t' + cat + '\t' + gene + '\t')
    
    for comparison in degDict:
        lfc = degDict[comparison][gene][1]
        pval = degDict[comparison][gene][4]
        fo.write(lfc + '\t' + pval + '\t')
        
    normcounts = expDict[gene]
    fo.write(normcounts + '\n')

fo.close()

In [6]:
!head -1 {expfile}

her19_1	her19_2	her19_3	WTHerm_1	WTHerm_2	WTHerm_3	Male_1	Male_2	Male_3
